In [ ]:
import requests

from random import randint

# dispersar
from cce import ClavePrivada
from ayudante import decodificar_base58, doble_sha256, little_endian_a_int, SIGHASH_ALL
from script import p2pkh_script, Script
from tx import TxIn, TxOut, Tx

passphrase = b'jimmy@programmingblockchain.com secret passphrase'
secreto = little_endian_a_int(double_sha256(passphrase))
priv = ClavePrivada(secreto=secreto)
mi_dire = priv.punto.dire(testnet=True)
print(mi_dire)
cantidad_destino = 0.1
comisión = 0.001

response = requests.get('https://testnet.blockchain.info/unspent?active={}'.format(mi_dire))
utxos = response.json()['unspent_outputs']
if (len(utxos) == 0):
    raise RuntimeError('sin outputs!')

direcciones_estudiantes = ''
# inicializa inputs
tx_ins = []
total = 0
for utxo in utxos:
    tx_previa = bytes.fromhex(utxo['tx_hash_big_endian'])
    indice_previo = utxo['tx_output_n']
    total += utxo['value']
    # crea un nuevo tx input
    tx_ins.append(TxIn(
            tx_previa=tx_previa,
            indice_previo=indice_previo,
            script_sig=Script([]),
            sequence=0xffffffff,
        ))
# inicializa outputs
tx_outs = []
cantidad_output = 0
for dire_destino in direcciones_estudiantes.split():
    # decodifica el hash160 desde la dirección destino
    h160 = decodificar_base58(dire_destino)
    # convierte hash160 a script p2pkh
    script_pubkey = script_p2pkh(h160)
    # convierte cantidad destino a satoshis (multiplica por 100 millones)
    satoshis_destino = int(cantidad_destino * 100000000)
    cantidad_output += satoshis_destino
    # crea un nuevo tx output para destino
    tx_outs.append(TxOut(
        cantidad=satoshis_destino,
        script_pubkey=script_pubkey,
    ))

satoshis_cambio = total - int(comisión*100000000) - cantidad_output
h160 = decodificar_base58(mi_dire)
script_pubkey = script_p2pkh(h160)
tx_outs.append(TxOut(cantidad=satoshis_cambio, script_pubkey=script_pubkey))

# crea la transacción
tx_obj = Tx(versión=1, tx_ins=tx_ins, tx_outs=tx_outs, locktime=0, testnet=True)

# ahora firma el input 0 con la clave privada usando SIGHASH_ALL usando firmar_input
for i in range(len(tx_obj.tx_ins)):
    tx_obj.firmar_input(i, priv, SIGHASH_ALL)

# COMPROBACIÓN: la comisión es razonable
if tx_obj.comisión() > 0.05*100000000 or tx_obj.comisión() <= 0:
    raise RuntimeError('Comprueba que el cambio sea razonable. La comisión es {}'.format(tx_obj.comisión()))

# serializa y hex()
print(tx_obj.serializar().hex())

In [ ]:
import requests

from random import randint

# dispersar
from cce import ClavePrivada
from ayudante import decodificar_base58, doble_sha256, little_endian_a_int, SIGHASH_ALL
from script import script_p2pkh, Script
from tx import TxIn, TxOut, Tx

passphrase = b'jimmy@programmingblockchain.com secret passphrase'

secreto = little_endian_a_int(double_sha256(passphrase))
priv = ClavePrivada(secreto=secreto)
mi_dire = priv.punto.dire(testnet=True)
cantidad_destino = 0.1
comisión = 0.001

response = requests.get('https://testnet.blockchain.info/unspent?active={}'.format(mi_dire))
utxos = response.json()['unspent_outputs']
if (len(utxos) == 0):
    raise RuntimeError('sin outputs!')

direcciones_estudiantes = ''
# inicializa inputs
tx_ins = []
total = 0
for utxo in utxos:
    tx_previa = bytes.fromhex(utxo['tx_hash_big_endian'])
    indice_previo = utxo['tx_output_n']
    total += utxo['value']
    # crea un nuevo tx input
    tx_ins.append(TxIn(
            tx_previa=tx_previa,
            indice_previo=indice_previo,
            script_sig=Script([]),
            sequence=0xffffffff,
        ))

# inicializa outputs
tx_outs = []
cantidad_output = 0
for dire_destino in direcciones_estudiantes.split():
    # decodifica el hash160 desde la dirección destino
    h160 = decodificar_base58(dire_destino)
    # convierte hash160 a script p2pkh
    script_pubkey = script_p2pkh(h160)
    # convierte cantidad destino a satoshis (multiplica por 100 millones)
    satoshis_destino = randint(5000000, 20000000)
    cantidad_output += satoshis_destino
    # crea un nuevo tx output para el destino
    tx_outs.append(TxOut(
        cantidad=satoshis_destino,
        script_pubkey=script_pubkey,
    ))

satoshis_cambio = total - int((comisión)*100000000) - cantidad_output

h160 = decodificar_base58(mi_dire)
script_pubkey = script_p2pkh(h160)
tx_outs.append(TxOut(cantidad=satoshis_cambio, script_pubkey=script_pubkey))

# crea la transacción
tx_obj = Tx(versión=1, tx_ins=tx_ins, tx_outs=tx_outs, locktime=0, testnet=True)

# ahora firma el input num 0 con la clave privada usando SIGHASH_ALL y firmar_input
for i in range(len(tx_obj.tx_ins)):
    tx_obj.firmar_input(i, priv, SIGHASH_ALL)

# COMPROBACIÓN: la comisión es razonable
if tx_obj.comisión() > 0.05*100000000 or tx_obj.comisión() <= 0:
    raise RuntimeError('Comrpueba que la cantidad de cambio sea razonable. La comisión es {}'.format(tx_obj.comisión()))

# serializa y hex()
print(tx_obj.serializar().hex())